# Sensor Exploration - Phase 1.3

**Feature**: 002-sensor-exploration  
**Author**: Luis Felipe Cordeiro Sena  
**Date**: 2025-11-21  

## Objectives

- **US3 (FR-014 to FR-019)**: LIDAR data analysis and visualization
- **US4 (FR-020 to FR-026)**: Camera RGB analysis and color detection

## Prerequisites

- Webots R2023b running with `IA_20252.wbt` loaded
- Python venv active with dependencies (numpy, matplotlib, opencv, scipy)
- YouBot controller integrated with this notebook

In [ ]:
# Setup
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

# Enable inline plotting
%matplotlib inline

# Add controller path
CONTROLLER_PATH = Path().absolute().parent / 'IA_20252' / 'controllers' / 'youbot'
sys.path.insert(0, str(CONTROLLER_PATH))

print(f"✅ Setup complete")
print(f"📂 Controller path: {CONTROLLER_PATH}")

## 1. LIDAR Analysis (FR-014 to FR-019)

### 1.1 Specifications (FR-015)

In [ ]:
def capture_lidar_specifications(lidar):
    """Extract and document LIDAR sensor specifications (FR-015)."""
    specs = {
        'horizontal_resolution': lidar.getHorizontalResolution(),
        'num_layers': lidar.getNumberOfLayers(),
        'fov': lidar.getFov(),
        'vertical_fov': lidar.getVerticalFov() if hasattr(lidar, 'getVerticalFov') else 0.0,
        'min_range': lidar.getMinRange(),
        'max_range': lidar.getMaxRange(),
    }
    specs['angular_resolution'] = specs['fov'] / (specs['horizontal_resolution'] - 1)
    return specs

# NOTE: Execute this after initializing robot in Webots
# from controller import Robot
# robot = Robot()
# lidar = robot.getDevice("lidar")
# lidar.enable(int(robot.getBasicTimeStep()))
# robot.step(int(robot.getBasicTimeStep()))
# lidar_specs = capture_lidar_specifications(lidar)
# print("LIDAR Specifications:", lidar_specs)

### 1.2 Data Capture (FR-014, FR-016)

In [ ]:
def capture_lidar_scan(lidar):
    """Capture single LIDAR scan (FR-014)."""
    range_image = lidar.getRangeImage()
    return np.array(range_image, dtype=np.float64)

def analyze_lidar_ranges(scans, specs):
    """Analyze LIDAR detection range capabilities (FR-016)."""
    all_scans = np.concatenate(scans)
    valid_readings = all_scans[np.isfinite(all_scans)]
    
    return {
        'observed_min_range': float(np.min(valid_readings)) if len(valid_readings) > 0 else 0.0,
        'observed_max_range': float(np.max(valid_readings)) if len(valid_readings) > 0 else 0.0,
        'mean_range': float(np.mean(valid_readings)) if len(valid_readings) > 0 else 0.0,
        'std_range': float(np.std(valid_readings)) if len(valid_readings) > 0 else 0.0,
        'valid_readings_pct': float(len(valid_readings) / len(all_scans) * 100) if len(all_scans) > 0 else 0.0,
    }

# Example execution:
# scans = [capture_lidar_scan(lidar) for _ in range(20)]
# range_analysis = analyze_lidar_ranges(scans, lidar_specs)
# print("Range Analysis:", range_analysis)

### 1.3 Visualization (FR-017)

In [ ]:
def visualize_lidar_polar(range_image, fov, max_range, title="LIDAR Scan"):
    """Generate polar plot of LIDAR scan (FR-017)."""
    num_points = len(range_image)
    angles = np.linspace(-fov/2, fov/2, num_points)
    
    # Replace inf with max_range for visualization
    distances = np.where(np.isinf(range_image), max_range, range_image)
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='polar')
    
    # Plot scan points
    scatter = ax.scatter(angles, distances, c=distances, cmap='RdYlGn_r', s=20, alpha=0.6)
    
    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
    cbar.set_label('Distance (m)', rotation=270, labelpad=20)
    
    # Labels
    ax.set_theta_zero_location('N')
    ax.set_title(title, va='bottom', fontsize=14, pad=20)
    ax.set_ylim(0, max_range)
    ax.grid(True)
    
    return fig

# Example:
# fig = visualize_lidar_polar(scans[0], lidar_specs['fov'], lidar_specs['max_range'])
# plt.savefig('../media/lidar_scans/lidar_scan_example.png', dpi=150, bbox_inches='tight')
# plt.show()

### 1.4 Obstacle Detection (FR-018)

In [ ]:
def identify_obstacles_lidar(range_image, fov, threshold=1.0):
    """Detect obstacles from LIDAR scan using distance threshold (FR-018)."""
    num_points = len(range_image)
    angles = np.linspace(-fov/2, fov/2, num_points)
    
    obstacles = []
    for i, (distance, angle) in enumerate(zip(range_image, angles)):
        if np.isfinite(distance) and distance < threshold:
            # Calculate confidence based on neighboring points
            neighbors = range_image[max(0, i-2):min(num_points, i+3)]
            valid_neighbors = neighbors[np.isfinite(neighbors)]
            confidence = len(valid_neighbors) / 5.0  # Up to 5 neighbors
            
            obstacles.append({
                'distance': float(distance),
                'angle': float(angle),
                'index': i,
                'confidence': confidence,
            })
    
    return obstacles

# Example:
# obstacles = identify_obstacles_lidar(scans[0], lidar_specs['fov'], threshold=1.0)
# print(f"Detected {len(obstacles)} obstacles within 1.0m")

## 2. Camera Analysis (FR-020 to FR-026)

### 2.1 Specifications (FR-021, FR-022)

In [ ]:
def capture_camera_specifications(camera):
    """Extract and document camera sensor specifications (FR-021, FR-022)."""
    specs = {
        'width': camera.getWidth(),
        'height': camera.getHeight(),
        'fov': camera.getFov(),
        'channels': 4,  # BGRA
        'has_recognition': camera.hasRecognition() if hasattr(camera, 'hasRecognition') else False,
    }
    return specs

def measure_camera_fps(camera, duration_sec=10):
    """Measure camera frame rate (FR-022)."""
    # NOTE: Requires robot.step() calls in loop
    # frame_count = 0
    # start_time = robot.getTime()
    # while robot.getTime() - start_time < duration_sec:
    #     robot.step(time_step)
    #     camera.getImage()  # Trigger frame capture
    #     frame_count += 1
    # fps = frame_count / duration_sec
    # return fps
    pass  # Placeholder

# Example:
# camera = robot.getDevice("camera")
# camera.enable(time_step)
# robot.step(time_step)
# camera_specs = capture_camera_specifications(camera)
# print("Camera Specifications:", camera_specs)

### 2.2 Frame Capture (FR-020, FR-023)

In [ ]:
def capture_camera_frame(camera):
    """Capture single camera frame and convert to RGB (FR-020)."""
    image_data = camera.getImage()
    if image_data is None:
        return None
    
    width = camera.getWidth()
    height = camera.getHeight()
    
    # Convert BGRA to RGB
    img = np.frombuffer(image_data, np.uint8).reshape((height, width, 4))
    img_rgb = img[:, :, [2, 1, 0]]  # BGR to RGB
    
    return img_rgb

def save_example_images(camera, robot, colors=['green', 'blue', 'red'], output_dir='../media/cube_examples'):
    """Capture and save example images of colored cubes (FR-023)."""
    os.makedirs(output_dir, exist_ok=True)
    saved_files = {}
    
    # NOTE: Manual positioning required to capture each color
    # For automation, use recognition API or position robot near known cubes
    
    print("⚠️ save_example_images requires manual execution:")
    print("   1. Position robot near green cube")
    print("   2. Run: frame = capture_camera_frame(camera)")
    print("   3. Save: cv2.imwrite('green_cube.png', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))")
    print("   4. Repeat for blue and red cubes")
    
    return saved_files

# Example:
# frame = capture_camera_frame(camera)
# plt.imshow(frame)
# plt.title("Camera Frame")
# plt.axis('off')
# plt.show()

### 2.3 Color Detection (FR-024)

In [ ]:
def detect_color_threshold(image_rgb, method='hsv'):
    """Classify cube color using threshold method (FR-024)."""
    # HSV thresholds (to be calibrated with real images)
    HSV_RANGES = {
        'green': ((40, 50, 50), (80, 255, 255)),
        'blue': ((90, 50, 50), (130, 255, 255)),
        'red_low': ((0, 50, 50), (10, 255, 255)),   # Red wraps around
        'red_high': ((170, 50, 50), (180, 255, 255)),
    }
    
    if method == 'hsv':
        # Convert to HSV
        img_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
        
        # Count pixels for each color
        pixel_counts = {}
        
        for color, (lower, upper) in HSV_RANGES.items():
            if color.startswith('red'):
                continue  # Handle red separately
            
            mask = cv2.inRange(img_hsv, np.array(lower), np.array(upper))
            pixel_counts[color] = int(np.sum(mask > 0))
        
        # Red color (combine low and high ranges)
        mask_red_low = cv2.inRange(img_hsv, np.array(HSV_RANGES['red_low'][0]), np.array(HSV_RANGES['red_low'][1]))
        mask_red_high = cv2.inRange(img_hsv, np.array(HSV_RANGES['red_high'][0]), np.array(HSV_RANGES['red_high'][1]))
        pixel_counts['red'] = int(np.sum(mask_red_low > 0) + np.sum(mask_red_high > 0))
        
        # Determine detected color
        total_pixels = image_rgb.shape[0] * image_rgb.shape[1]
        max_color = max(pixel_counts, key=pixel_counts.get)
        confidence = pixel_counts[max_color] / total_pixels
        
        detected_color = max_color if confidence > 0.05 else 'unknown'  # 5% threshold
        
        return {
            'detected_color': detected_color,
            'confidence': confidence,
            'pixel_counts': pixel_counts,
            'method': method,
        }
    else:
        raise ValueError(f"Method '{method}' not implemented")

# Example:
# result = detect_color_threshold(frame, method='hsv')
# print(f"Detected: {result['detected_color']} (confidence: {result['confidence']:.1%})")

### 2.4 Accuracy Evaluation (FR-025)

In [ ]:
def evaluate_color_detection_accuracy(test_images):
    """Evaluate threshold method on labeled test set (FR-025)."""
    if len(test_images) == 0:
        return {
            'overall_accuracy': 0.0,
            'per_color_accuracy': {},
            'confusion_matrix': np.zeros((3, 3)),
            'num_samples': 0,
        }
    
    colors = ['green', 'blue', 'red']
    confusion = np.zeros((3, 3), dtype=int)
    
    correct = 0
    for img, ground_truth in test_images:
        result = detect_color_threshold(img, method='hsv')
        predicted = result['detected_color']
        
        if predicted == ground_truth:
            correct += 1
        
        # Update confusion matrix
        if ground_truth in colors and predicted in colors:
            gt_idx = colors.index(ground_truth)
            pred_idx = colors.index(predicted)
            confusion[gt_idx, pred_idx] += 1
    
    overall_accuracy = correct / len(test_images)
    
    # Per-color accuracy
    per_color_accuracy = {}
    for i, color in enumerate(colors):
        total_color = np.sum(confusion[i, :])
        correct_color = confusion[i, i]
        per_color_accuracy[color] = correct_color / total_color if total_color > 0 else 0.0
    
    return {
        'overall_accuracy': overall_accuracy,
        'per_color_accuracy': per_color_accuracy,
        'confusion_matrix': confusion,
        'num_samples': len(test_images),
    }

# Example:
# test_images = [(frame1, 'green'), (frame2, 'blue'), (frame3, 'red')]
# accuracy = evaluate_color_detection_accuracy(test_images)
# print(f"Overall Accuracy: {accuracy['overall_accuracy']:.1%}")
# print(f"Per-color: {accuracy['per_color_accuracy']}")

## 3. Results Summary

### 3.1 LIDAR Findings

**Specifications:**
- Horizontal resolution: [VALUE] points
- FOV: [VALUE] rad
- Max range: [VALUE] m

**Range Analysis:**
- Observed min: [VALUE] m
- Observed max: [VALUE] m
- Valid readings: [VALUE]%

**Obstacle Detection:**
- Threshold: 1.0m
- Obstacles detected: [VALUE]

### 3.2 Camera Findings

**Specifications:**
- Resolution: [WIDTH] × [HEIGHT]
- FOV: [VALUE] rad
- Estimated FPS: [VALUE]

**Color Detection:**
- Method: HSV thresholding
- Overall accuracy: [VALUE]%
- Green accuracy: [VALUE]%
- Blue accuracy: [VALUE]%
- Red accuracy: [VALUE]%

### 3.3 Recommendations for Phase 2

**LIDAR Processing:**
- Use polar coordinates for neural network input
- Consider PointNet architecture for 3D point cloud processing
- Implement clustering (DBSCAN) for obstacle grouping

**Camera Processing:**
- HSV thresholding baseline: [ACCURACY]%
- Consider CNN for improved robustness (lighting variations)
- Augment training data with rotations, brightness adjustments

**Sensor Fusion:**
- LIDAR provides distance, camera provides color
- Combine for robust cube detection and localization
- Fuzzy logic can integrate sensor confidences